# Skyler's Salary Prediction Portfolio

## DEFINE

## Make salary predictions on each role with help from mean-squared-error (MSE) models

In [2]:
''' this script defines a problem and discovers the data'''

__author__ = 'Skyler Bullard, Business Analyst'
__contactinfo__ = 'skyler.bullard.r@gmail.com'

### import everything we might need

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st

### load the data

In [4]:
path = '/Users/sky/Users/Skyler/DSDJ/Personal Portfolio/salarypredictionportfolio/data/'

test_features = pd.read_csv( open(path + 'test_features.csv') )
train_features = pd.read_csv( open(path + 'train_features.csv') )
train_salaries = pd.read_csv( open(path + 'train_salaries.csv') )

### examine the data

In [5]:
test_features.head(10)

jobId companyId         jobType       degree    major industry  \
0  JOB1362685407687    COMP33         MANAGER  HIGH_SCHOOL     NONE   HEALTH   
1  JOB1362685407688    COMP13          JUNIOR         NONE     NONE     AUTO   
2  JOB1362685407689    COMP10             CTO      MASTERS  BIOLOGY   HEALTH   
3  JOB1362685407690    COMP21         MANAGER  HIGH_SCHOOL     NONE      OIL   
4  JOB1362685407691    COMP36          JUNIOR     DOCTORAL  BIOLOGY      OIL   
5  JOB1362685407692    COMP40             CTO      MASTERS  COMPSCI  FINANCE   
6  JOB1362685407693    COMP32          SENIOR      MASTERS  COMPSCI  SERVICE   
7  JOB1362685407694    COMP11             CEO    BACHELORS  BIOLOGY  SERVICE   
8  JOB1362685407695    COMP39         MANAGER     DOCTORAL  PHYSICS   HEALTH   
9  JOB1362685407696    COMP22  VICE_PRESIDENT         NONE     NONE     AUTO   

   yearsExperience  milesFromMetropolis  
0               22                   73  
1               20                   47  
2               17                    9  
3               14                   96  
4               10                   44  
5                6                   23  
6                6                   32  
7                9                   73  
8                6                   78  
9               23                   97

In [6]:
train_features.head(10)

jobId companyId         jobType       degree      major  \
0  JOB1362684407687    COMP37             CFO      MASTERS       MATH   
1  JOB1362684407688    COMP19             CEO  HIGH_SCHOOL       NONE   
2  JOB1362684407689    COMP52  VICE_PRESIDENT     DOCTORAL    PHYSICS   
3  JOB1362684407690    COMP38         MANAGER     DOCTORAL  CHEMISTRY   
4  JOB1362684407691     COMP7  VICE_PRESIDENT    BACHELORS    PHYSICS   
5  JOB1362684407692    COMP15         MANAGER     DOCTORAL    COMPSCI   
6  JOB1362684407693    COMP15             CFO         NONE       NONE   
7  JOB1362684407694    COMP24          JUNIOR    BACHELORS  CHEMISTRY   
8  JOB1362684407695    COMP20         JANITOR  HIGH_SCHOOL       NONE   
9  JOB1362684407696    COMP41  VICE_PRESIDENT    BACHELORS  CHEMISTRY   

    industry  yearsExperience  milesFromMetropolis  
0     HEALTH               10                   83  
1        WEB                3                   73  
2     HEALTH               10                   38  
3       AUTO                8                   17  
4    FINANCE                8                   16  
5    FINANCE                2                   31  
6     HEALTH               23                   24  
7  EDUCATION                9                   70  
8  EDUCATION                1                   54  
9       AUTO               17                   68

In [7]:
train_salaries.head(10)

jobId  salary
0  JOB1362684407687     130
1  JOB1362684407688     101
2  JOB1362684407689     137
3  JOB1362684407690     142
4  JOB1362684407691     163
5  JOB1362684407692     113
6  JOB1362684407693     178
7  JOB1362684407694      73
8  JOB1362684407695      31
9  JOB1362684407696     104

### use .info() to see length and datatypes

In [8]:
test_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [9]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [10]:
train_salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   jobId   1000000 non-null  object
 1   salary  1000000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 15.3+ MB


### check for duplicates

In [11]:
test_features.duplicated().sum()

0

In [12]:
train_features.duplicated().sum()

0

In [13]:
train_salaries.duplicated().sum()

0

### identify numerical and categorical variables

In [14]:
train_features.columns

Index(['jobId', 'companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis'],
      dtype='object')

In [15]:
numeric_cols = [ 'yearsExperience', 'milesFromMetropolis' ]

In [16]:
categorical_cols = [ 'jobId', 'companyId', 'jobType', 'degree', 'major', 'industry' ]

## summarize numerical and categorical variables separately

In [17]:
train_features.describe( include = [ np.number ] )

yearsExperience  milesFromMetropolis
count   1000000.000000       1000000.000000
mean         11.992386            49.529260
std           7.212391            28.877733
min           0.000000             0.000000
25%           6.000000            25.000000
50%          12.000000            50.000000
75%          18.000000            75.000000
max          24.000000            99.000000

In [18]:
train_features.describe( include = [ 'O' ] )

jobId companyId  jobType       degree    major industry
count            1000000   1000000  1000000      1000000  1000000  1000000
unique           1000000        63        8            5        9        7
top     JOB1362684407687    COMP39   SENIOR  HIGH_SCHOOL     NONE      WEB
freq                   1     16193   125886       236976   532355   143206

## merge training data into a single dataframe

In [19]:
train_df = pd.merge( train_features , train_salaries , on = "jobId" )

In [20]:
del train_features
del train_salaries

In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
 8   salary               1000000 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 76.3+ MB


In [22]:
train_df.head()

jobId companyId         jobType       degree      major  \
0  JOB1362684407687    COMP37             CFO      MASTERS       MATH   
1  JOB1362684407688    COMP19             CEO  HIGH_SCHOOL       NONE   
2  JOB1362684407689    COMP52  VICE_PRESIDENT     DOCTORAL    PHYSICS   
3  JOB1362684407690    COMP38         MANAGER     DOCTORAL  CHEMISTRY   
4  JOB1362684407691     COMP7  VICE_PRESIDENT    BACHELORS    PHYSICS   

  industry  yearsExperience  milesFromMetropolis  salary  
0   HEALTH               10                   83     130  
1      WEB                3                   73     101  
2   HEALTH               10                   38     137  
3     AUTO                8                   17     142  
4  FINANCE                8                   16     163

In [40]:
train_df.salary.describe()

count    1000000.000000
mean         116.061818
std           38.717936
min            0.000000
25%           88.000000
50%          114.000000
75%          141.000000
max          301.000000
Name: salary, dtype: float64

In [37]:
#print the modal value of the training target
st.mode( train_df.salary )

ModeResult(mode=array([108]), count=array([10467]))

## use IQR rule to find potential outliers

In [43]:
#sort the training target
train_df.sort_values( 'salary' )

jobId companyId         jobType       degree        major  \
828156  JOB1362685235843    COMP40  VICE_PRESIDENT      MASTERS  ENGINEERING   
652076  JOB1362685059763    COMP25             CTO  HIGH_SCHOOL         NONE   
495984  JOB1362684903671    COMP34          JUNIOR         NONE         NONE   
30559   JOB1362684438246    COMP44          JUNIOR     DOCTORAL         MATH   
816129  JOB1362685223816    COMP42         MANAGER     DOCTORAL  ENGINEERING   
...                  ...       ...             ...          ...          ...   
911747  JOB1362685319434     COMP3             CEO    BACHELORS     BUSINESS   
293221  JOB1362684700908    COMP44             CEO     DOCTORAL  ENGINEERING   
572777  JOB1362684980464    COMP60             CEO      MASTERS  ENGINEERING   
685562  JOB1362685093249    COMP47             CTO      MASTERS  ENGINEERING   
400611  JOB1362684808298    COMP45             CFO     DOCTORAL  ENGINEERING   

       industry  yearsExperience  milesFromMetropolis  salary  
828156      WEB                3                   29       0  
652076     AUTO                6                   60       0  
495984      OIL                1                   25       0  
30559      AUTO               11                    7       0  
816129  FINANCE               18                    6       0  
...         ...              ...                  ...     ...  
911747  FINANCE               24                   28     294  
293221      OIL               19                   18     294  
572777      OIL               22                    1     298  
685562      OIL               24                    0     301  
400611      OIL               23                    3     301  

[1000000 rows x 9 columns]

In [45]:
#The interquartile range (IQR) tells you the range of the middle half of your dataset.
#You can define outliers that fall outside of "fences"
stat_IQR = train_df.salary.describe()
IQR = stat_IQR[ '75%' ] - stat_IQR[ '25%' ]
upper_fence = stat_IQR[ '75%' ] + ( 1.5 * IQR )
lower_fence = stat_IQR[ '25%' ] - ( 1.5 * IQR )
print( 'The upper and lower bounds for what may be outliers are {} and {}.'.format( upper_fence , lower_fence ) )

The upper and lower bounds for what may be outliers are 220.5 and 8.5.


## outlier detection

In [46]:
#check past lower fence
train_df[ train_df.salary < 8.5 ]

jobId companyId         jobType       degree        major  \
30559   JOB1362684438246    COMP44          JUNIOR     DOCTORAL         MATH   
495984  JOB1362684903671    COMP34          JUNIOR         NONE         NONE   
652076  JOB1362685059763    COMP25             CTO  HIGH_SCHOOL         NONE   
816129  JOB1362685223816    COMP42         MANAGER     DOCTORAL  ENGINEERING   
828156  JOB1362685235843    COMP40  VICE_PRESIDENT      MASTERS  ENGINEERING   

       industry  yearsExperience  milesFromMetropolis  salary  
30559      AUTO               11                    7       0  
495984      OIL                1                   25       0  
652076     AUTO                6                   60       0  
816129  FINANCE               18                    6       0  
828156      WEB                3                   29       0

In [47]:
#check past upper fence
train_df.loc[ train_df.salary > 220.5 , 'jobType' ].value_counts()

CEO               3227
CFO               1496
CTO               1488
VICE_PRESIDENT     603
MANAGER            217
SENIOR              66
JUNIOR              20
Name: jobType, dtype: int64